# 10-1. Farneback 알고리즘으로 광류 추정하기

In [1]:
import numpy as np
import cv2 as cv
import sys

def draw_OpticalFlow(img, flow, step=16):
    for y in range(step//2, frame.shape[0], step):
        for x in range(step//2, frame.shape[1], step):
            dx, dy = flow[y,x].astype(np.int)
            if (dx*dx + dy*dy) > 1:
                cv.line(img, (x,y), (x+dx, y+dy), (0,0,255), 2)     # 큰 모션이 있는 곳은 빨간색
            else:
                cv.line(img, (x,y), (x+dx, y+dx), (0,255,0), 2)
    
cap = cv.VideoCapture(0,  cv.CAP_DSHOW)     # 카메라와 연결 시도
if not cap.isOpened(): sys.exit('카메라 연결 실패')

prev = None

while(1):
    ret, frame = cap.read()                 # 비디오를 구성하는 프레임 획득
    if not ret: sys('프레임 획득에 실패하여 루프를 나갑니다.')

    if prev is None:                        # 첫 프레임이면 광류 계산 없이 prev만 설정
        prev = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        continue

    curr = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prev, curr, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    draw_OpticalFlow(frame, flow)
    cv.imshow('Optical flow', frame)

    prev = curr

    key = cv.waitKey(1)     # 1밀리초 동안 키보드 입력 기다림
    if key == ord('q'):     # 'q' 키가 들어오면 루프를 빠져나감
        break

cap.release()               # 카메라와 연결을 끊음
cv.destroyAllWindows()

# 10-2. KLT 추적 알고리즘으로 물체 추적하기

In [2]:
import numpy as np
import cv2 as cv

cap = cv.VideoCapture('slow_traffic_small.mp4')

feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15,15), maxLevel=2, criteria=(cv.TERM_CRITERIA_EPS|cv.TERM_CRITERIA_COUNT,10,0.03))

color = np.random.randint(0, 255, (100, 3))

ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

mask = np.zeros_like(old_frame)

while(1):
    ret, frame = cap.read()
    if not ret: break

    new_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    p1, match, err = cv.calcOpticalFlowPyrLK(old_gray, new_gray, p0, None, **lk_params)

    if p1 is not None:
        good_new = p1[match==1]
        good_old = p0[match==1]

    for i in range(len(good_new)):
        a, b = int(good_new[i][0]), int(good_new[i][1])
        c, d = int(good_old[i][0]), int(good_old[i][1])
        mask = cv.line(mask, (a,b), (c,d), color[i].tolist(), 2)
        frame = cv.circle(frame, (a,b), 5, color[i].tolist(), -1)

    img = cv.add(frame, mask)
    cv.imshow('LTK tracker', img)
    cv.waitKey(30)

    old_gray = new_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv.destroyAllWindows()

# 10-3. SORT로 사람 추적하기

In [5]:
import numpy as np
import cv2 as cv
import sys

# [9-1 예제와 동일 (YOLOv3로 물체를 검출하는 함수)]
def construct_yolo_v3():
    f = open('coco_names.txt', 'r')
    class_names = [line.strip() for line in f.readlines()]

    model = cv.dnn.readNet('yolov3.weights', 'yolov3.cfg')
    layer_names = model.getLayerNames()
    out_layers = [layer_names[i-1] for i in model.getUnconnectedOutLayers()]        # getUnconnectedOutLayers()는 출력 레이어의 인덱스를 반환

    return model, out_layers, class_names

def yolo_detect(img, yolo_model, out_layers):
    height, width = img.shape[0], img.shape[1]
    test_img = cv.dnn.blobFromImage(img, 1.0/256, (448, 448), (0, 0, 0), swapRB=True)   # img, scalefactor, size, mean, swapRB

    yolo_model.setInput(test_img)
    output3 = yolo_model.forward(out_layers)

    box, conf, id = [], [], []          # 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:        # 신뢰도가 50% 이상인 경우만 취함
                center_x, center_y = int(vec85[0]*width), int(vec85[1]*height)
                w, h = int(vec85[2]*width), int(vec85[3]*height)
                x, y = int(center_x-w/2), int(center_y-h/2)
                box.append([x, y, x+w, y+h])
                conf.append(float(confidence))
                id.append(class_id)

    ind = cv.dnn.NMSBoxes(box, conf, 0.5, 0.4)        # box, conf, score_threshold, nms_threshold
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]
    return objects

model, out_layers, class_names = construct_yolo_v3()        # YOLO 모델 생성
colors = np.random.uniform(0, 255, size=(100,3))            # 100개 색으로 트랙 구분

from sort import Sort

sort = Sort()

cap = cv.VideoCapture(0, cv.CAP_DSHOW)
if not cap.isOpened(): sys.exit('카메라 연결 실패')

while True:
    ret, frame = cap.read()
    if not ret: sys.exit('프레임 획득에 실패하여 루프를 나갑니다.')

    res = yolo_detect(frame, model, out_layers)
    persons = [res[i] for i in range(len(res)) if res[i][5]==0]

    if len(persons) == 0:
        tracks = sort.update()
    else:
        tracks = sort.update(np.array(persons))

    for i in range(len(tracks)):
        x1, y1, x2, y2, track_id = tracks[i].astype(int)
        cv.rectangle(frame, (x1,y1), (x2,y2), colors[track_id], 2)
        cv.putText(frame, str(track_id), (x1+10, y1+40), cv.FONT_HERSHEY_PLAIN, 3, colors[track_id], 2)

    cv.imshow('Person tracking by SORT', frame)

    key = cv.waitKey(1)
    if key == ord('q'): break

cap.release()
cv.destroyAllWindows()